In [1]:
from flask import Flask, render_template_string, request
import pandas as pd
from fuzzywuzzy import fuzz, process

# Initialize the Flask app
app = Flask(__name__)

# Load the data from Excel
file_path = r"C:\Users\Saleel01\Downloads\Leeds-20241110T220642Z-001\Leeds\Work Done\New folder\datasets\Datasetfinaldivya12_11.xlsx"
excel_data = pd.ExcelFile(file_path)

# Load each sheet into a DataFrame
disease_bodyfn_df = excel_data.parse('30-10-DISEASE VS BODYFN')
bodyfn_assistive_df = excel_data.parse('31-10 BODYFN vs assistive techn')
bodyfn_home_icf_df = excel_data.parse('31-10 BODYFN vs HOME ICF')

# Helper function to find the closest match using fuzzy matching
def find_closest_match(input_text, text_list):
    match = process.extractOne(input_text, text_list, scorer=fuzz.partial_ratio)
    return match[0] if match and match[1] >= 50 else None

# Function to retrieve details based on disease input
def get_disease_details(disease_name):
    disease_list = disease_bodyfn_df['Disease'].str.lower().tolist()
    matched_disease = find_closest_match(disease_name.lower(), disease_list)
    
    if matched_disease is None:
        return f"No information found for disease: {disease_name}", [], [], []

    disease_info = disease_bodyfn_df[disease_bodyfn_df['Disease'].str.lower() == matched_disease]
    symptoms = disease_info['Symptoms'].values[0].split(', ') if not disease_info.empty else []
    assistive_mapping, home_function_mapping, synonym_mapping = [], [], []

    all_symptoms_assistive = bodyfn_assistive_df['Symptoms'].str.lower().tolist()
    all_symptoms_home = bodyfn_home_icf_df['Symptoms'].str.lower().tolist()

    for symptom in symptoms:
        matched_symptom_assistive = find_closest_match(symptom, all_symptoms_assistive)
        tech_info = bodyfn_assistive_df[bodyfn_assistive_df['Symptoms'].str.lower() == matched_symptom_assistive]
        assistive_tech = tech_info['Assistive Technology'].values[0] if not tech_info.empty else "No assistive technology found"
        assistive_mapping.append((symptom, assistive_tech))

        matched_symptom_home = find_closest_match(symptom, all_symptoms_home)
        home_info = bodyfn_home_icf_df[bodyfn_home_icf_df['Symptoms'].str.lower() == matched_symptom_home]
        home_functions = ", ".join(home_info['Home functions'].dropna().tolist()) or "No home functions found"
        home_function_mapping.append((symptom, home_functions))

        synonym_mapping.append((symptom, matched_symptom_assistive, matched_symptom_home))

    return matched_disease, assistive_mapping, home_function_mapping, synonym_mapping

# HTML template with styling for the web interface
template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>HOMEAI-ENABLE</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            background-image: url('https://via.placeholder.com/1600x900');
            background-size: cover;
            background-attachment: fixed;
            color: #ffffff;
            margin: 0;
            padding: 0;
        }
        .container {
            width: 80%;
            margin: 0 auto;
            background: rgba(0, 0, 0, 0.7);
            padding: 20px;
            border-radius: 8px;
        }
        h1 {
            text-align: center;
            color: #00ffcc;
            font-size: 3em;
            margin-top: 20px;
        }
        form {
            text-align: center;
            margin: 20px 0;
        }
        label {
            font-size: 1.2em;
            color: #00ffcc;
        }
        input[type="text"] {
            padding: 10px;
            font-size: 1em;
            width: 60%;
            margin: 10px;
            border-radius: 5px;
            border: 1px solid #00ffcc;
        }
        button {
            padding: 10px 20px;
            font-size: 1em;
            background-color: #00ffcc;
            color: #000;
            border: none;
            border-radius: 5px;
            cursor: pointer;
        }
        button:hover {
            background-color: #00cc99;
        }
        table {
            width: 100%;
            margin: 20px 0;
            border-collapse: collapse;
            background-color: #ffffff;
            color: #000;
        }
        th, td {
            padding: 12px;
            text-align: left;
            border: 1px solid #dddddd;
        }
        th {
            background-color: #00cc99;
            color: white;
        }
        footer {
            text-align: center;
            margin-top: 20px;
            color: #00ffcc;
            font-size: 0.9em;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>HOMEAI-ENABLE</h1>
        <form method="POST">
            <label for="disease_name">Enter Disease Name:</label>
            <input type="text" id="disease_name" name="disease_name" required>
            <button type="submit">Submit</button>
        </form>
        <br>
        {% if message %}
            <p>{{ message }}</p>
        {% endif %}
        {% if disease %}
            <h2>Matched Disease: {{ disease }}</h2>
            <h3>Symptoms</h3>
            <ul>
                {% for symptom in symptoms %}
                    <li>{{ symptom }}</li>
                {% endfor %}
            </ul>
            <h3>Assistive Technologies</h3>
            <table>
                <tr><th>Symptom</th><th>Assistive Technology</th></tr>
                {% for item in assistive_mapping %}
                    <tr><td>{{ item[0] }}</td><td>{{ item[1] }}</td></tr>
                {% endfor %}
            </table>
            <h3>Home Functions</h3>
            <table>
                <tr><th>Symptom</th><th>Home Functions</th></tr>
                {% for item in home_function_mapping %}
                    <tr><td>{{ item[0] }}</td><td>{{ item[1] }}</td></tr>
                {% endfor %}
            </table>
            <h3>Synonym Mapping Details</h3>
            <table>
                <tr><th>Input Symptom</th><th>Matched Symptom (Assistive)</th><th>Matched Symptom (Home)</th></tr>
                {% for item in synonym_mapping %}
                    <tr><td>{{ item[0] }}</td><td>{{ item[1] }}</td><td>{{ item[2] }}</td></tr>
                {% endfor %}
            </table>
        {% endif %}
    </div>
    <footer>
        <p>&copy; 2023 HOMEAI-ENABLE. All rights reserved.</p>
    </footer>
</body>
</html>
"""

# Define the main page route
@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        disease_name = request.form.get('disease_name')
        matched_disease, assistive_mapping, home_function_mapping, synonym_mapping = get_disease_details(disease_name)
        if isinstance(matched_disease, str) and "No information found" in matched_disease:
            return render_template_string(template, message=matched_disease)
        return render_template_string(
            template, 
            disease=matched_disease.capitalize(),
            symptoms=[symptom[0] for symptom in assistive_mapping],
            assistive_mapping=assistive_mapping,
            home_function_mapping=home_function_mapping,
            synonym_mapping=synonym_mapping
        )
    return render_template_string(template)


In [2]:
import threading
import webbrowser

# Function to run the Flask app
def run_app():
    app.run(debug=False, use_reloader=False)

# Function to open the browser automatically
def open_browser():
    webbrowser.open("http://127.0.0.1:5000")  # Open the browser immediately

# Start Flask app in a background thread to avoid blocking
def start_flask():
    threading.Thread(target=run_app).start()
    threading.Timer(1, open_browser).start()  # Wait 1 second before opening the browser

# Start the Flask server and open the browser
start_flask()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [22/Nov/2024 21:13:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2024 21:13:01] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [22/Nov/2024 21:14:17] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2024 21:15:13] "POST / HTTP/1.1" 200 -
